In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

In [22]:
Feb = pd.read_csv('../KW_Mask_Geolocated/1/31_mask_geoloc.csv')

Feb.shape

(58, 8)

In [23]:
Feb.head(20)

,id,text,created_at,urls,profile_location,geotag_location,geotag_country,is_USA
0,1223036013586632705,The #coronarvirus isn't only killing people; i...,2020-01-31 00:11:23+00:00,NaN,"Chicago, IL",NaN,NaN,1
1,1223036276556742656,The #coronarvirus isn't only killing people; i...,2020-01-31 00:12:25+00:00,NaN,occupied Kalapuya land//OR,NaN,NaN,1
2,1223036424863330306,"Finally, WHO has declared the new coronavirus ...",2020-01-31 00:13:01+00:00,NaN,USA,NaN,NaN,1
3,1223040989885083653,Chinese people massing in a crowd WITHOUT a ma...,2020-01-31 00:31:09+00:00,['https://banned.video/watch?id=5e3338b0fd3c03...,USA,NaN,NaN,1
4,1223041493134446592,The #coronarvirus isn't only killing people; i...,2020-01-31 00:33:09+00:00,NaN,United States,NaN,NaN,1
5,1223042645855014912,The #coronarvirus isn't only killing people; i...,2020-01-31 00:37:44+00:00,NaN,"Saco, ME",NaN,NaN,1
6,1223044823164358656,The #coronarvirus isn't only killing people; i...,2020-01-31 00:46:23+00:00,NaN,"San Antonio, TX",NaN,NaN,1
7,1223045194645295104,*Coronavirus death toll hits 170* “Omf everyon...,2020-01-31 00:47:52+00:00,NaN,"San Francisco, CA",NaN,NaN,1
8,1223045263385878528,The #coronarvirus isn't only killing people; i...,2020-01-31 00:48:08+00:00,NaN,"Rochester, NY",NaN,NaN,1
9,1223048043232157697,@btsarmyshan Rn I’m more concerned about the f...,2020-01-31 00:59:11+00:00,NaN,United States,NaN,NaN,1


# Pattern Match For States

In [24]:
matcher = Matcher(nlp.vocab)

state_abbv = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", 
    "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", 
    "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", 
    "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan",
    "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", 
    "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", 
    "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", 
    "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", 
    "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

states = state_abbv + state_names

pattern = [{"LEMMA": {"IN": states}}]

matcher.add("State", [pattern])

# Subset tweets to ones with viable locations

In [26]:
#doc = nlp(Feb.profile_location.loc[0])
#doc = nlp('Paris, Texas')

inds = []

for location in Feb.profile_location:
    doc = nlp(location)
    matches = matcher(doc)

    # If there are matches go with first state mention (Sorry Kansas City)
    if len(matches) > 0:
        _, start, end = matches[0]

        span = doc[start:end]
        inds.append(1)

        #print(span.text)
    
    else:
        inds.append(0)
        

Feb[np.array(inds,dtype=bool)]

,id,text,created_at,urls,profile_location,geotag_location,geotag_country,is_USA
0,1223036013586632705,The #coronarvirus isn't only killing people; i...,2020-01-31 00:11:23+00:00,NaN,"Chicago, IL",NaN,NaN,1
7,1223045194645295104,*Coronavirus death toll hits 170* “Omf everyon...,2020-01-31 00:47:52+00:00,NaN,"San Francisco, CA",NaN,NaN,1
8,1223045263385878528,The #coronarvirus isn't only killing people; i...,2020-01-31 00:48:08+00:00,NaN,"Rochester, NY",NaN,NaN,1
10,1223049257906651137,*Coronavirus death toll hits 170* “Omf everyon...,2020-01-31 01:04:00+00:00,NaN,"Long Beach, CA",NaN,NaN,1
12,1223053456686178305,The #coronarvirus isn't only killing people; i...,2020-01-31 01:20:41+00:00,NaN,"Alpharetta, GA",NaN,NaN,1
15,1223060391602917377,Chinese people massing in a crowd WITHOUT a ma...,2020-01-31 01:48:15+00:00,['https://banned.video/watch?id=5e3338b0fd3c03...,"Pennsylvania, USA",NaN,NaN,1
16,1223062587018887168,The #coronarvirus isn't only killing people; i...,2020-01-31 01:56:58+00:00,NaN,"Chicago, IL",NaN,NaN,1
19,1223063898196250624,The #coronarvirus isn't only killing people; i...,2020-01-31 02:02:11+00:00,NaN,"Miami, FL ♿",NaN,NaN,1
21,1223074368395321346,"Lets panic: Only 5 U.S. cases, but recovering ...",2020-01-31 02:43:47+00:00,['https://twitter.com/realDonaldTrump/status/1...,"Las Vegas, NV",NaN,NaN,1
22,1223074598993850368,The #coronarvirus isn't only killing people; i...,2020-01-31 02:44:42+00:00,NaN,"Los Angeles, CA",NaN,NaN,1


In [35]:
doc = nlp(Feb.profile_location.loc[5])


for token in doc:
    # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            # token.shape_, token.is_alpha, token.is_stop)
    print(token.text, token.lemma_, token.pos_, token.tag_, token.ent_type_)

Saco Saco PROPN NNP 
, , PUNCT , 
ME I PRON PRP 


In [33]:
doc = nlp("")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [71]:
alc_keywords_short = ['alcohol', 'alcoholic', 'alcoholism', 'drink', 'beer', 'liquor', 'wine', 'spirits']


alc_keywords = [
    'alcohol',
    'alcoholic',
    'alcoholism',
    'drinking',
    'beer',
    'liquor',
    'wine',
    'spirits',
    'quit alcohol',
    'leave alcohol',
    'quit drinking',
    'alcohol detox',
    'stop alcohol'
    'stop drinking',
    'alcohol treatment',
    'liquor store near me',
    'buy alcohol online',
    'make alcohol at home',
    'benzodiazepines',
    'alcohol withdrawal',
    'alcohol addiction',
    'alcohol dependence',
    'alcohol pregnant',
    'alcohol pregnancy',
    'alcohol anxiety',
    'alcohol depression',
    'alcohol sale',
    'homemade alcohol',
    'alcohol from sanitizer',
    'DUI',
    'alcoholics anonymous',
    'liver disease',
    'alcoholic liver disease',
    'hangover',
    'hangover cures',
    'beer of the month club',
    'wine of the month club',
    'best whiskey',
    'best scotch',
    'best vodka',
    'best gin',
    'best tequila',
    'best rum',
    'best brandy',
    'too much alcohol',
    'alcohol vaccine']

In [69]:
doc = nlp("Beer")

for token in doc:
    print(token.text, token.lemma_)

fine fine
wines wine


# Now with entire month

In [5]:
months_tweets_alc = pd.read_pickle('month_tweets_alc.pkl')

print(months_tweets_alc.shape)
months_tweets_alc.head()

(3222, 7)


,id,text,created_at,urls,profile_location,geotag_location,geotag_country
2640,1223495848723509248,Apparently the corona virus can be killed with...,2020-02-01 06:38:36+00:00,NaN,"Sydney, New South Wales",NaN,NaN
16857,1226662644402343936,Yes and let's not forget the Lysol anti Virus ...,2020-02-10 00:22:19+00:00,['https://twitter.com/drrandywax/status/122665...,"Kentucky, USA",NaN,NaN
25818,1226669623845634053,Q: Does drinking alcohol prevent the new #coro...,2020-02-10 00:50:03+00:00,['https://twitter.com/i/web/status/12259866065...,Singapore,NaN,NaN
28062,1226671360593711104,Reduce risk of #coronavirus infection ➡️ Clean...,2020-02-10 00:56:57+00:00,['https://twitter.com/i/web/status/12266713605...,Kuala Lumpur,NaN,NaN
29118,1226672182903934976,corona virus: dies in high temperature and alc...,2020-02-10 01:00:13+00:00,NaN,toronto,NaN,NaN


In [98]:

for tweet in months_tweets_alc_geo.text.head():
    print(tweet)


The #coronavirus has sparked some strange rumors. Does it come from Corona beer? Should you protect your dog with a face mask? Can drinking bleach prevent an infection? My latest article for @Healthline gets to the bottom of these weird myths. https://t.co/jWOCv013x7
The #coronavirus has sparked some strange rumors. Does it come from Corona beer? Should you protect your dog with a face mask? Can drinking bleach prevent an infection? My latest article for @Healthline gets to the bottom of these weird myths. https://t.co/jWOCv013x7
So now that corona virus has been confirmed in Nigeria! * Apart from bathing with alcohol * Chewing chloroquine like biscuits * pple wearing face mask like subzero What other dramatic experiments should we be expecting!!?? #coronavirusnigeria https://t.co/TT0DrL3Nyx
@GoodBlogPosts6 Masks don’t help with Corona virus. Corona beer? Yes
@gojonewsnow Yes!! I heard that the Corona Beer is mostly where the coronavirus is spread...... What the beer doesn't know is th